# NF26/AI07 - TD1 : Initiation à PySpark



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Introduction à PySpark

L'objectif de ce TD est de s'initier à PySpark sur une base de données simulées.
Pour cela nous nous appuyerons sur le lien ci-dessous :
- https://spark.apache.org/docs/latest/api/python/index.html

**Introduction.** Durant les 15 premières minutes de ce TD, rechercher sur le site ci-dessus ou sur tout autre site des informations pour comprendre qu'est-ce que PySpark, quel est son lien avec Spark, quels sont ses avantages et pourquoi il est apprécié dans la communauté.

**Installation de PySpark.** Compiler la commande ci-dessous pour installer PySpark.

In [ ]:
pip install pyspark

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, date

import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [ ]:
spark = SparkSession.builder.getOrCreate()

## Information concernant la base de données `foo.csv`

La base de données `foo.csv` est une base de données simulées et contient des informations générales concernant les patients qui sont arrivés dans un hopital durant l'année 2023.

La commande ci-dessous permet de lire la base de données `foo.csv`:

In [ ]:
psdf = ps.read_csv('/content/drive/MyDrive/Colab Notebooks/foo.csv')
psdf.head(10)

/usr/local/lib/python3.11/dist-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,KeyConsult,KeyPatient,Name,FirstName,Date_In,Pathology,Date_Out
0,KeyConsult9163,KeyPatient5557,Name5557,FistName5557,2023-11-29 23:00:00,Pathology89,2023-12-10 23:00:00
1,KeyConsult9164,KeyPatient2469,Name2469,FistName2469,2023-11-29 23:00:00,Pathology28,2023-11-29 23:00:00
2,KeyConsult9165,KeyPatient20346,Name20346,FistName20346,2023-11-29 23:00:00,Pathology36,2023-11-29 23:00:00
3,KeyConsult9166,KeyPatient43536,Name43536,FistName43536,2023-11-29 23:00:00,Pathology49,2023-11-30 23:00:00
4,KeyConsult9167,KeyPatient346,Name346,FistName346,2023-11-29 23:00:00,Pathology39,2023-11-29 23:00:00
5,KeyConsult9168,KeyPatient48522,Name48522,FistName48522,2023-11-29 23:00:00,Pathology18,2023-11-30 23:00:00
6,KeyConsult9169,KeyPatient33615,Name33615,FistName33615,2023-11-29 23:00:00,Pathology30,2023-11-30 23:00:00
7,KeyConsult9170,KeyPatient16913,Name16913,FistName16913,2023-11-29 23:00:00,Pathology53,2023-11-30 23:00:00
8,KeyConsult9171,KeyPatient30785,Name30785,FistName30785,2023-11-29 23:00:00,Pathology7,2023-12-01 23:00:00
9,KeyConsult9172,KeyPatient9418,Name9418,FistName9418,2023-11-29 23:00:00,Pathology50,2023-11-29 23:00:00


In [ ]:
psdf.dtypes

,0
KeyConsult,object
KeyPatient,object
Name,object
FirstName,object
Date_In,datetime64[ns]
Pathology,object
Date_Out,datetime64[ns]


In [ ]:
sdf = psdf.to_spark(index_col='index')
type(sdf)

pyspark.sql.dataframe.DataFrame

In [ ]:
from pyspark.sql.functions import *

## Exercices :

**Question 1.** Proposer un code permettant de visualiser les données triées par date d'entrée dans l'hôpital. On pourra afficher seulement les 20 premières dates.

In [ ]:
# Réponse :
sdf.orderBy("Date_In").show()
# sdf.filter(year('Date_In')==2023).sort(sdf.Date_In.asc()).show()


+-----+------------+---------------+---------+-------------+-------------------+-----------+-------------------+
|index|  KeyConsult|     KeyPatient|     Name|    FirstName|            Date_In|  Pathology|           Date_Out|
+-----+------------+---------------+---------+-------------+-------------------+-----------+-------------------+
| 9167| KeyConsult0|KeyPatient46375|Name46375|FistName46375|2022-12-31 23:00:00|Pathology15|2023-01-02 23:00:00|
| 9187|KeyConsult20| KeyPatient6941| Name6941| FistName6941|2022-12-31 23:00:00|Pathology13|2022-12-31 23:00:00|
| 9168| KeyConsult1|KeyPatient13621|Name13621|FistName13621|2022-12-31 23:00:00|Pathology16|2022-12-31 23:00:00|
| 9169| KeyConsult2|  KeyPatient729|  Name729|  FistName729|2022-12-31 23:00:00|Pathology23|2023-01-02 23:00:00|
| 9170| KeyConsult3|KeyPatient25954|Name25954|FistName25954|2022-12-31 23:00:00|Pathology23|2023-01-04 23:00:00|
| 9171| KeyConsult4|KeyPatient39686|Name39686|FistName39686|2022-12-31 23:00:00|Pathology59|2022

**Question 2.** Proposer un code permettant de visualiser le nombre d'entrées quotidiennes dans l'hôpital durant l'année 2023.

In [ ]:
# Réponse :
nb_entries = sdf.filter(year('Date_In')==2023).groupBy("Date_In").agg(countDistinct('KeyPatient').alias('Number_Entries')).orderBy("Date_In")
nb_entries.show()
# nb_entries = sdf.filter(year('Date_In')==2023).groupBy([year('Date_In'), month('Date_In'), day('Date_In')]).count().show()



+-------------------+--------------+
|            Date_In|Number_Entries|
+-------------------+--------------+
|2023-01-01 23:00:00|            20|
|2023-01-02 23:00:00|            33|
|2023-01-03 23:00:00|            25|
|2023-01-04 23:00:00|            27|
|2023-01-05 23:00:00|            18|
|2023-01-06 23:00:00|            28|
|2023-01-07 23:00:00|            28|
|2023-01-08 23:00:00|            33|
|2023-01-09 23:00:00|            21|
|2023-01-10 23:00:00|            17|
|2023-01-11 23:00:00|            27|
|2023-01-12 23:00:00|            25|
|2023-01-13 23:00:00|            28|
|2023-01-14 23:00:00|            24|
|2023-01-15 23:00:00|            32|
|2023-01-16 23:00:00|            22|
|2023-01-17 23:00:00|            20|
|2023-01-18 23:00:00|            25|
|2023-01-19 23:00:00|            25|
|2023-01-20 23:00:00|            17|
+-------------------+--------------+
only showing top 20 rows



**Question 3.** Quel a été le nombre maximal et minimal d'entrées quotidiennes dans l'hôpital durant l'année 2023 ?

In [ ]:
# Réponse :
nb_entries.select(min(nb_entries.Number_Entries), max(nb_entries.Number_Entries)).show()


+-------------------+-------------------+
|min(Number_Entries)|max(Number_Entries)|
+-------------------+-------------------+
|                 15|                 43|
+-------------------+-------------------+



**Question 4.** Proposer un code permettant de visualiser si certains patients sont venus plusieurs fois dans l'hôpital durant l'année 2023 (et si oui combien de fois).

In [ ]:
# Réponse :
nb_patient = sdf.filter(year('Date_in')==2023).groupBy('KeyPatient').agg(count('KeyPatient').alias("nb_visite"))
nb_patient.filter("nb_visite > 1").show()
nb_patient.filter("nb_visite > 1").count()

+---------------+---------+
|     KeyPatient|nb_visite|
+---------------+---------+
|KeyPatient31576|        2|
|KeyPatient33121|        2|
| KeyPatient7996|        2|
|  KeyPatient507|        2|
| KeyPatient1351|        2|
| KeyPatient2381|        2|
|KeyPatient48986|        2|
|KeyPatient43089|        2|
| KeyPatient7496|        3|
|KeyPatient28497|        2|
|KeyPatient37257|        2|
|KeyPatient33618|        2|
|KeyPatient40044|        2|
|KeyPatient45099|        2|
| KeyPatient4580|        2|
|KeyPatient27080|        2|
|KeyPatient30589|        2|
|KeyPatient21147|        2|
|KeyPatient33128|        2|
|KeyPatient21413|        2|
+---------------+---------+
only showing top 20 rows



861

**Question 5.** Quel a été le nombre maximal et minimal d'entrées par patient dans l'hôpital durant l'année 2023 ?

In [ ]:
# Réponse :
nb_patient.select(min(nb_patient.nb_visite), max(nb_patient.nb_visite)).show()


+--------------+--------------+
|min(nb_visite)|max(nb_visite)|
+--------------+--------------+
|             1|             4|
+--------------+--------------+



**Question 6.** Proposer un code permettant de visualiser quels patients ont été diagnostiqués de la `Pathology15` durant l'année 2023.

In [ ]:
# Réponse :
sdf.filter(sdf.Pathology=='Pathology15').drop_duplicates(['KeyPatient']).select(sdf.KeyPatient).show()

+---------------+
|     KeyPatient|
+---------------+
|KeyPatient34677|
|KeyPatient16820|
| KeyPatient1865|
|KeyPatient48965|
|KeyPatient25400|
|KeyPatient41068|
|KeyPatient30917|
|KeyPatient32640|
|KeyPatient27659|
|KeyPatient28718|
|  KeyPatient519|
| KeyPatient7483|
|KeyPatient43436|
|KeyPatient17066|
| KeyPatient7782|
|KeyPatient16801|
|KeyPatient28119|
|KeyPatient19720|
| KeyPatient6846|
|KeyPatient44520|
+---------------+
only showing top 20 rows



**Question 7.** Proposer un code permettant de visualiser quels ont été les 10 premiers patients à être diagnostiqués de la `Pathology15` durant l'année 2023.

In [ ]:
# Réponse :
sdf.filter((year('Date_In')==2023) & (sdf.Pathology=='Pathology15')).orderBy('Date_In').show(10)

+-----+--------------+---------------+---------+-------------+-------------------+-----------+-------------------+
|index|    KeyConsult|     KeyPatient|     Name|    FirstName|            Date_In|  Pathology|           Date_Out|
+-----+--------------+---------------+---------+-------------+-------------------+-----------+-------------------+
| 9214|  KeyConsult47|KeyPatient18512|Name18512|FistName18512|2023-01-02 23:00:00|Pathology15|2023-01-03 23:00:00|
| 9312| KeyConsult145| KeyPatient3785| Name3785| FistName3785|2023-01-05 23:00:00|Pathology15|2023-01-06 23:00:00|
| 9488| KeyConsult321|KeyPatient19197|Name19197|FistName19197|2023-01-12 23:00:00|Pathology15|2023-01-12 23:00:00|
| 9684| KeyConsult517| KeyPatient4575| Name4575| FistName4575|2023-01-20 23:00:00|Pathology15|2023-01-21 23:00:00|
| 9707| KeyConsult540|KeyPatient18840|Name18840|FistName18840|2023-01-22 23:00:00|Pathology15|2023-01-23 23:00:00|
| 8462| KeyConsult961|KeyPatient28031|Name28031|FistName28031|2023-02-06 23:00:0

**Question 8.** Proposer un code permettant de visualiser s'il existe des patients qui ont été diagnostiqués plusieurs fois de la `Pathology17` durant l'année 2023 (et si oui combien de fois au maximum).

In [ ]:
# Réponse :
sdf.filter(sdf.Pathology=='Pathology17').groupBy('KeyPatient').count().filter("count > 1").show()

+--------------+-----+
|    KeyPatient|count|
+--------------+-----+
|KeyPatient6629|    2|
+--------------+-----+

